In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from helper_functions_pipe_testing import *
from sklearn.metrics import  f1_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import *

In [2]:
#Make static BioPy Library from UniProt library
uniprot_filepath='UniProt/Human_Proteome_221216.xlsx'
uniprot_dat=pd.read_excel(uniprot_filepath,header=0)
PROT_cleaned_data = clean_up_data_biopy(uniprot_dat)
PROT_cleaned_data.to_excel('Input_data/BioPython_data/Human_biopyCalcs.xlsx',index=False)

C:\Users\kmp95\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


20404


In [12]:
df=PROT_cleaned_data.drop_duplicates()

In [14]:
df.to_excel('Input_data/BioPython_data/Human_biopyCalcs.xlsx',index=False)

In [3]:
uniprot_dat.shape

(20404, 10)

In [4]:
PROT_cleaned_data.shape

(20626, 39)

In [23]:
df2= pd.concat([PROT_cleaned_data['Entry'],uniprot_dat['Entry']])

In [25]:
df2.unique()

array(['A0A087X1C5', 'A0A0B4J2F0', 'A0A0B4J2F2', ..., 'P59797', 'P63302',
       'Q8IZQ5'], dtype=object)

In [11]:
# PROT_cleaned_data[~PROT_cleaned_data['Entry'].isin(uniprot_dat['Entry'])]
# PROT_cleaned_data.duplicated(subset=['Entry'],keep=False)
PROT_cleaned_data[PROT_cleaned_data['Entry'].duplicated()==True]
# PROT_cleaned_data['Entry'] not in uniprot_dat['Entry']
# uniprot_dat2=uniprot_dat[~uniprot_dat['Entry'].isin(PROT_cleaned_data['Entry'])]
# uniprot_dat2.to_excel('UniProt/Human_missed_proteinsinbiopy.xlsx',index=False)
# PROT_cleaned_data.shape
# uniprot_dat['Entry'].shape
# list = set(PROT_cleaned_data.Entry) | set(uniprot_dat.Entry)

,Entry,Sequence,Length,Mass,frac_aa_A,frac_aa_C,frac_aa_D,frac_aa_E,frac_aa_F,frac_aa_G,...,flexibility_var,flexibility_max,flexibility_min,flexibility_median,isoelectric_point,secondary_structure_fraction_helix,secondary_structure_fraction_turn,secondary_structure_fraction_sheet,secondary_structure_fraction_disordered,gravy
170,B7ZAQ6,MSFLIDSSIMITSQILFFGFGWLFFMRQLFKDYEIRQYVVQVIFSV...,455,52917,0.037363,0.010989,0.026374,0.039560,0.096703,0.050549,...,0.000831,1.056155,0.923893,0.979202,9.339615,0.454945,0.184615,0.246154,0.114286,0.556264
172,P0CG08,MSFLIDSSIMITSQILFFGFGWLFFMRQLFKDYEIRQYVVQVIFSV...,455,52917,0.037363,0.010989,0.026374,0.039560,0.096703,0.050549,...,0.000831,1.056155,0.923893,0.979202,9.339615,0.454945,0.184615,0.246154,0.114286,0.556264
1874,P01593,MDMRVPAQLLGLLLLWLSGARCDIQMTQSPSSLSASVGDRVTITCQ...,117,12848,0.059829,0.025641,0.068376,0.017094,0.025641,0.068376,...,0.000713,1.066821,0.942690,0.997720,4.527817,0.299145,0.290598,0.222222,0.188034,-0.205983
1876,P01594,MDMRVPAQLLGLLLLWLSGARCDIQMTQSPSSLSASVGDRVTITCQ...,117,12848,0.059829,0.025641,0.068376,0.017094,0.025641,0.068376,...,0.000713,1.066821,0.942690,0.997720,4.527817,0.299145,0.290598,0.222222,0.188034,-0.205983
1878,P01597,MDMRVPAQLLGLLLLWLRGARCDIQMTQSPSSLSASVGDRVTITCR...,117,12737,0.068376,0.025641,0.042735,0.008547,0.025641,0.068376,...,0.000704,1.066821,0.939524,0.996768,8.905485,0.290598,0.307692,0.222222,0.179487,-0.130769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20398,P0DMW5,MAASAALSAAAAAAALSGLAVRLSRSAAARGSYGAFCKGLTRTLLT...,78,8389,0.230769,0.012821,0.012821,0.012821,0.064103,0.051282,...,0.000373,1.017488,0.941417,0.981845,11.253811,0.346154,0.179487,0.423077,0.051282,0.683333
20403,P0DPP9,MDKEVPKGSPREPALNIKKSDKSFKRKKPTENVLIFLINRQLGRHR...,59,6991,0.016949,0.000000,0.067797,0.050847,0.033898,0.033898,...,0.001273,1.076643,0.949321,1.016714,10.432548,0.288136,0.254237,0.203390,0.254237,-0.844068
20405,Q3ZM63,MDKEVPKGSPREPALNIKKSDKSFKRKKPTENVLIFLINRQLGRHR...,59,6991,0.016949,0.000000,0.067797,0.050847,0.033898,0.033898,...,0.001273,1.076643,0.949321,1.016714,10.432548,0.288136,0.254237,0.203390,0.254237,-0.844068
20412,P0DV73,MMARRDPKSWAKRLVRAQTLQKQRRAPVGPRAPPPDEEDPRLKCKN...,464,49759,0.105603,0.021552,0.034483,0.056034,0.017241,0.068966,...,0.000657,1.091571,0.954810,1.013071,9.953677,0.168103,0.327586,0.262931,0.241379,-0.771336


In [6]:
# df=uniprot_dat[uniprot_dat['Entry'].isin(uniprot_dat2)]
uniprot_dat2

,Entry,Entry Name,Protein names,Gene Names,Organism,Length,Sequence,Mass,Gene Ontology (GO),Gene Ontology (biological process)


In [9]:
df

,Entry,Entry Name,Protein names,Gene Names,Organism,Length,Sequence,Mass,Gene Ontology (GO),Gene Ontology (biological process)


In [4]:
test2 = clean_up_data_biopy(uniprot_dat2)

                                                  0         1         2   \
0  MVAMAAGPSGCLVPAFGLRLLLATVLQAVSAFGAEFSSEACRELGF...  0.084848  0.054545   

         3         4         5         6    7         8         9   ...  \
0  0.048485  0.078788  0.060606  0.078788  0.0  0.030303  0.054545  ...   

         26        27        28        29        30        31        32  \
0  0.000638  1.061583  0.944488  1.000369  4.956269  0.327273  0.224242   

         33        34        35  
0  0.333333  0.115152  0.132727  

[1 rows x 36 columns]
                                                  0         1         2   \
0  MVAMAAGPSGCLVPAFGLRLLLATVLQAVSAFGAEFSSEACRELGF...  0.084848  0.054545   
0  MCAARLAAAAAAAQSVYAFSARPLAGGEPVSLGSLRGKVLLIENVA...  0.118227  0.029557   

         3         4         5         6         7         8         9   ...  \
0  0.048485  0.078788  0.060606  0.078788  0.000000  0.030303  0.054545  ...   
0  0.039409  0.064039  0.054187  0.083744  0.009852  0.029557  0.

In [6]:
print(test2)

Empty DataFrame
Columns: [Entry, Sequence, Length, Mass, frac_aa_A, frac_aa_C, frac_aa_D, frac_aa_E, frac_aa_F, frac_aa_G, frac_aa_H, frac_aa_I, frac_aa_K, frac_aa_L, frac_aa_M, frac_aa_N, frac_aa_P, frac_aa_Q, frac_aa_R, frac_aa_S, frac_aa_T, frac_aa_V, frac_aa_W, frac_aa_Y, molecular_weight, aromaticity, instability_index, flexibility_mean, flexibility_std, flexibility_var, flexibility_max, flexibility_min, flexibility_median, isoelectric_point, secondary_structure_fraction_helix, secondary_structure_fraction_turn, secondary_structure_fraction_sheet, secondary_structure_fraction_disordered, gravy]
Index: []

[0 rows x 39 columns]


In [27]:
list

{'A4D1N5',
 'P57081',
 'Q9H490',
 'O96006',
 'Q8TCU5',
 'Q13033',
 'Q9H2H0',
 'Q99811',
 'A0A2Z4LIS9',
 'O00258',
 'P07996',
 'Q9BQQ3',
 'P98182',
 'P0CG32',
 'Q8N3S3',
 'O95900',
 'Q8NDX5',
 'P05089',
 'Q8WXD2',
 'O94952',
 'P55285',
 'A8MUM7',
 'O43314',
 'A0A0C4DH33',
 'Q6UWF5',
 'O95503',
 'P23763',
 'Q9UK33',
 'Q5VV67',
 'A0A1B0GV85',
 'Q9Y467',
 'H3BR10',
 'Q8NG11',
 'Q8N448',
 'Q9BUP0',
 'A2RUT3',
 'Q6ICG8',
 'Q16585',
 'P05186',
 'Q9H478',
 'Q96BA8',
 'Q9H000',
 'Q8IVL5',
 'Q96GX1',
 'Q9C0B5',
 'Q969F0',
 'P48444',
 'Q9BR84',
 'P37173',
 'Q99929',
 'Q9Y2Y0',
 'Q9Y6H8',
 'O75348',
 'Q02446',
 'P04001',
 'P11509',
 'Q9HB75',
 'Q9BUV8',
 'Q09MP3',
 'Q53H80',
 'Q4G0T1',
 'Q5XKR4',
 'P17600',
 'P34949',
 'Q86VZ6',
 'Q14CZ7',
 'Q5T0Z8',
 'Q8TEK3',
 'Q6P2I3',
 'Q8N2U9',
 'P61020',
 'Q9Y6J6',
 'O14867',
 'Q9Y463',
 'Q5ST30',
 'Q8NCB2',
 'Q6PIZ9',
 'P42773',
 'A0A087WXM9',
 'P0C7W9',
 'Q2M3A8',
 'Q96GT9',
 'Q7RTS3',
 'Q9P217',
 'P61769',
 'Q52LJ0',
 'Q8N8I6',
 'Q9C0B7',
 'Q86US8',
 'Q9N

In [8]:
#determine any proteins missing from NSP database that are in uniprot database and export to excel spreadsheet
NSPfilePath='NetsurfP_Proteomes/nsp_human_proteome.xlsx'
NSP_data=pd.read_excel(NSPfilePath)
df=pd.merge(uniprot_dat,NSP_data,on='Entry',how='outer',indicator=True)
df=df[df['_merge']=='left_only']
df.to_excel('NetsurfP_Proteomes/human_proteome_missingproteins.xlsx')
#use this spreadsheet to find list of accession numbers, get FASTA files from uniprot
#run data through NSP (if small enough just do by hand)

In [4]:
#combine NSP data
combined_list=[]
combined_list.append(pd.read_excel('Input_data/NetSurfP_data/Bovine_Mouse_proteome_complete.xlsx'))
combined_list.append(pd.read_excel('Input_data/NetSurfP_data/Human_Proteome.xlsx'))
combined=pd.concat(combined_list, ignore_index=True)
combined.to_excel('Input_data/NetSurfP_data/Combined.xlsx', index=False)

In [3]:
#Combine Biopython Data
combined_list=[]
combined_list.append(pd.read_excel('Input_data/BioPython_data/Bovine_Mouse_proteome_biopyCalcs.xlsx'))
combined_list.append(pd.read_excel('Input_data/BioPython_data/Human_biopyCalcs.xlsx'))
combined=pd.concat(combined_list, ignore_index=True)
combined.to_excel('Input_data/BioPython_data/Combined_biopyCalcs.xlsx', index=False)

In [4]:
combined.shape

(43977, 39)

In [7]:
check=pd.read_excel('Input_data/BioPython_data/Bovine_Mouse_proteome_biopyCalcs.xlsx')
check.shape

(23573, 39)

In [2]:
#combine Controls
control_files=['controls_BALF2.xlsx','controls_FBS2.xlsx','controls_human_internal.xlsx']
control_dir='Input_data/Proteomic data/'
import glob
combined_list=[]
for file in control_files:
    combined_list.append(pd.read_excel(control_dir+file))
combined= pd.concat(combined_list, ignore_index=True)
combined.dropna(how='all', axis=1,inplace=True)
combined.to_excel(control_dir+'controls_combined_internal_human.xlsx',index=False)